In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;

modelPath = '../models/VGG16_opt/'
modelName = 'run1.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="0"

if not os.path.exists(modelPath):
    os.makedirs(modelPath)

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)

We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [16]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
    base_model = VGG16(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))

    dropout = {{uniform(0.5,1)}};
    layers = {{choice([0,1,2])}};
    dontFreeze = {{choice(list(range(5+1)))}};
    batchSize = {{choice([16,64,256])}};
    
    print()
    print('dropout=',dropout)
    print('layers=',layers)
    print('dontFreeze=',dontFreeze)
    print('batchSize=',batchSize)
    print()
    
    stepsPerEpoch = round( len(X_train)/batchSize );
    
    # Add a new top layer
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    if layers>=2:
        x = Dense(1024,activation='relu')(x)
    if layers>=1:
        x = Dense(512,activation='relu')(x)
    # in any case:
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)-dontFreeze):
        base_model.layers[i].trainable = False

    # predetermined optimizer
    lr=0.00020389590556056983;
    beta_1=0.9453158868247398;
    beta_2=0.9925872692991417;
    decay=0.000821336141287018;
    adam = keras.optimizers.Adam(lr=lr,beta_1=beta_1,beta_2=beta_2,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));


    # data augmentation & fitting
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True);
    
    model.fit_generator(
        datagen.flow(X_train,Y_train,batch_size=batchSize),
        steps_per_epoch=stepsPerEpoch,
        epochs=150,
        verbose=1,
        validation_data=(X_valid,Y_valid),
        workers=2,
        shuffle=True,
        callbacks=callbacks_list)
#     model.fit(X_train, Y_train,
#       epochs=100,
#       batch_size = batchSize,
#       validation_data=(X_valid, Y_valid),
#       verbose=1,
#       callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [17]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='VGG16Optimization_1')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications.vgg16 import VGG16
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import cv2
except:
   

100%|██████████| 10222/10222 [00:30<00:00, 330.53it/s]


Splitting into training/validation
Creating model

dropout= 0.6064002165637792
layers= 2
dontFreeze= 3
batchSize= 64

Epoch 1/150
112/112 [==============================] - 30s 264ms/step - loss: 4.8055 - acc: 0.0073 - val_loss: 4.7833 - val_acc: 0.0108
Epoch 2/150
112/112 [==============================] - 25s 221ms/step - loss: 4.7843 - acc: 0.0101 - val_loss: 4.7840 - val_acc: 0.0153
Epoch 3/150
112/112 [==============================] - 25s 220ms/step - loss: 4.7758 - acc: 0.0144 - val_loss: 4.7663 - val_acc: 0.0137
Epoch 4/150
112/112 [==============================] - 25s 221ms/step - loss: 4.7706 - acc: 0.0158 - val_loss: 4.7538 - val_acc: 0.0147
Epoch 5/150
112/112 [==============================] - 25s 219ms/step - loss: 4.7507 - acc: 0.0168 - val_loss: 4.7477 - val_acc: 0.0140
Epoch 6/150
112/112 [==============================] - 25s 220ms/step - loss: 4.7437 - acc: 0.0155 - val_loss: 4.7288 - val_acc: 0.0170
Epoch 7/150
112/112 [==============================] - 25s 221ms/s

112/112 [==============================] - 25s 220ms/step - loss: 3.6109 - acc: 0.1424 - val_loss: 3.4664 - val_acc: 0.1702
Epoch 61/150
112/112 [==============================] - 25s 220ms/step - loss: 3.6021 - acc: 0.1473 - val_loss: 3.4701 - val_acc: 0.1725
Epoch 62/150
112/112 [==============================] - 25s 221ms/step - loss: 3.5819 - acc: 0.1490 - val_loss: 3.4740 - val_acc: 0.1705
Epoch 63/150
112/112 [==============================] - 25s 221ms/step - loss: 3.5731 - acc: 0.1515 - val_loss: 3.4506 - val_acc: 0.1738
Epoch 64/150
112/112 [==============================] - 25s 220ms/step - loss: 3.5669 - acc: 0.1540 - val_loss: 3.4437 - val_acc: 0.1767
Epoch 65/150
112/112 [==============================] - 25s 220ms/step - loss: 3.5570 - acc: 0.1520 - val_loss: 3.4397 - val_acc: 0.1813
Epoch 66/150
112/112 [==============================] - 25s 220ms/step - loss: 3.5572 - acc: 0.1561 - val_loss: 3.4509 - val_acc: 0.1728
Epoch 67/150
112/112 [==============================] 

Epoch 120/150
112/112 [==============================] - 25s 220ms/step - loss: 3.1997 - acc: 0.2123 - val_loss: 3.2598 - val_acc: 0.2136
Epoch 121/150
112/112 [==============================] - 25s 220ms/step - loss: 3.1868 - acc: 0.2267 - val_loss: 3.2967 - val_acc: 0.2119
Epoch 122/150
112/112 [==============================] - 25s 221ms/step - loss: 3.2114 - acc: 0.2133 - val_loss: 3.3038 - val_acc: 0.2132
Epoch 123/150
112/112 [==============================] - 25s 221ms/step - loss: 3.2013 - acc: 0.2158 - val_loss: 3.2250 - val_acc: 0.2224
Epoch 124/150
112/112 [==============================] - 25s 220ms/step - loss: 3.1652 - acc: 0.2206 - val_loss: 3.2658 - val_acc: 0.2172
Epoch 125/150
112/112 [==============================] - 25s 220ms/step - loss: 3.1805 - acc: 0.2153 - val_loss: 3.2428 - val_acc: 0.2172
Epoch 126/150
112/112 [==============================] - 25s 220ms/step - loss: 3.1609 - acc: 0.2233 - val_loss: 3.2863 - val_acc: 0.2139
Epoch 127/150
112/112 [===========

28/28 [==============================] - 20s 709ms/step - loss: 4.7753 - acc: 0.0178 - val_loss: 4.7028 - val_acc: 0.0323
Epoch 12/150
28/28 [==============================] - 20s 709ms/step - loss: 4.7582 - acc: 0.0219 - val_loss: 4.6935 - val_acc: 0.0355
Epoch 13/150
28/28 [==============================] - 20s 710ms/step - loss: 4.7549 - acc: 0.0196 - val_loss: 4.6850 - val_acc: 0.0355
Epoch 14/150
28/28 [==============================] - 20s 709ms/step - loss: 4.7389 - acc: 0.0208 - val_loss: 4.6760 - val_acc: 0.0368
Epoch 15/150
28/28 [==============================] - 20s 708ms/step - loss: 4.7387 - acc: 0.0243 - val_loss: 4.6668 - val_acc: 0.0375
Epoch 16/150
28/28 [==============================] - 20s 710ms/step - loss: 4.7272 - acc: 0.0208 - val_loss: 4.6573 - val_acc: 0.0404
Epoch 17/150
28/28 [==============================] - 20s 710ms/step - loss: 4.7064 - acc: 0.0274 - val_loss: 4.6437 - val_acc: 0.0437
Epoch 18/150
28/28 [==============================] - 20s 709ms/step

Epoch 72/150
28/28 [==============================] - 20s 710ms/step - loss: 4.3975 - acc: 0.0541 - val_loss: 4.1512 - val_acc: 0.1004
Epoch 73/150
28/28 [==============================] - 20s 710ms/step - loss: 4.4041 - acc: 0.0598 - val_loss: 4.1498 - val_acc: 0.0988
Epoch 74/150
28/28 [==============================] - 20s 713ms/step - loss: 4.3949 - acc: 0.0581 - val_loss: 4.1466 - val_acc: 0.1040
Epoch 75/150
28/28 [==============================] - 20s 710ms/step - loss: 4.3735 - acc: 0.0601 - val_loss: 4.1402 - val_acc: 0.1047
Epoch 76/150
28/28 [==============================] - 20s 710ms/step - loss: 4.3811 - acc: 0.0630 - val_loss: 4.1336 - val_acc: 0.1040
Epoch 77/150
28/28 [==============================] - 20s 712ms/step - loss: 4.3761 - acc: 0.0598 - val_loss: 4.1318 - val_acc: 0.1030
Epoch 78/150
28/28 [==============================] - 20s 707ms/step - loss: 4.3751 - acc: 0.0592 - val_loss: 4.1270 - val_acc: 0.1040
Epoch 79/150
28/28 [==============================] - 2

100%|██████████| 10222/10222 [00:39<00:00, 259.69it/s]


Splitting into training/validation


MemoryError: 

In [1]:
X_train, Y_train, X_test, Y_test = data()
val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

NameError: name 'data' is not defined

In [ ]:
best_model.save(modelPath);
best_model.summary()

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
# preds = model.predict(x_test, verbose=1)

In [ ]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)